# GIN
\
スカラー形式の式
$$
\pmb{h}_v^{(l+1)} = \text{MLP}^{(l+1)} \left(\left( 1 + \epsilon^{(l+1)} \right) \pmb{h}_v^{(l)} + \sum_{u \in \mathcal{N}(v) } \pmb{h}_u^{(l)}  \right)
$$
行列形式の式
$$
\mathbf{H}^{(l+1)} = \text{MLP}^{(l+1)} \left( \mathbf{A} + \left( 1 + \epsilon^{(l+1)} \right) \mathbf{I} \right) \mathbf{H}^{(l)}
$$

参考文献 :\
「How Powerful are Graph Neural Networks?」 \
https://arxiv.org/pdf/1810.00826 \
「PyG / torch_geometric.nn / conv.GINConv」 \
https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GINConv.html

# 準備

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import networkx as nx

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch_geometric.typing import Adj
import torch.nn.init as init

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx
from torch_geometric import datasets

from sklearn.manifold import TSNE

from typing import Any, List, Optional, Tuple

%matplotlib inline

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def train(epoche,
          data,
          optimizer,
          criterion,
          device,
          net,
          val = False
          ):

    optimizer.state_dict()['state'] = {}

    data = data.to(device)
    for i in range(epoche):
        net.train() # 訓練フェーズ
        optimizer.zero_grad()
        out = net(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])

        pred = out[data.train_mask].argmax(dim=1)
        correct = (pred == data.y[data.train_mask]).sum().item()
        total = data.train_mask.sum().item()
        train_acc = correct / total

        if val:
            val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
            val_pred = out[data.val_mask].argmax(dim=1)
            val_correct = (val_pred == data.y[data.val_mask]).sum().item()
            val_total = data.val_mask.sum().item()
            val_acc = val_correct / val_total
        loss.backward()
        optimizer.step()

        if val:
            print(f'Epoch: {i+1} \n Train loss: {loss:.5f}, Train acc:{train_acc:.5f}, Val loss: {val_loss:.5f}, Val acc:{val_acc:.5f}')
        else:
            print(f'Epoch: {i+1} \n Train loss: {loss}, Train acc:{train_acc:.5f}')

In [3]:
# デバイスの確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# PyTorch ベース

In [4]:
class GINlayer(nn.Module):
    def __init__(
        self,
        in_d: int,
        hidden_d: list,
        train_eps: bool = True,
        ):
        super().__init__()

        num_MLP_layer = len(hidden_d)

        self.mlp = torch.nn.ModuleList()

        # 学習パラメータの設定
        if train_eps:
            self.eps = nn.Parameter(torch.empty(()))

        else:
            self.register_buffer('eps', torch.tensor(0.0))

        # MLP の設定
        for i in range(num_MLP_layer):
            if i == 0:
                self.mlp.append(nn.Linear(in_d, hidden_d[i]))

            else:
                self.mlp.append(nn.BatchNorm1d(hidden_d[i - 1]))
                self.mlp.append(nn.ReLU())
                self.mlp.append(nn.Linear(hidden_d[i-1], hidden_d[i]))

        if train_eps:
            # パラメータの初期化
            self.reset_parameters()

    # 初期化関数
    def reset_parameters(self):
        init.normal_(self.eps)

    def forward(
        self,
        x: Tensor,
        edge_index: Adj
        ):

        num_nodes = x.shape[0]
        # 隣接行列を作る
        A = torch.zeros((num_nodes, num_nodes)).to(device)
        A[edge_index[0], edge_index[1]] = 1
        # 単位行列の作成
        I = torch.eye(num_nodes).to(device)

        eps_plus_one = 1 + self.eps
        new_A = A + (eps_plus_one * I)

        new_H = torch.matmul(new_A, x)

        for layer in self.mlp:
            new_H = layer(new_H)

        return new_H

In [5]:
# グラフ畳み込みネットワークの定義
class GIN(torch.nn.Module):
    def __init__(self, in_d_1, hidden_d_1, in_d_2, hidden_d_2):
        super().__init__()

        self.conv1 = GINlayer(in_d_1, hidden_d_1, train_eps=False)
        self.conv2 = GINlayer(in_d_2, hidden_d_2)

        self.bn = nn.BatchNorm1d(in_d_2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = self.bn(x)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        return x

In [6]:
dataset = datasets.Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0]

in_d_1 = dataset.num_node_features
hidden_d_1 = [in_d_1]

in_d_2 = hidden_d_1[-1]
hidden_d_2 = [1024, dataset.num_classes]

net = GIN(in_d_1, hidden_d_1, in_d_2, hidden_d_2).to(device)
print(net)

GIN(
  (conv1): GINlayer(
    (mlp): ModuleList(
      (0): Linear(in_features=1433, out_features=1433, bias=True)
    )
  )
  (conv2): GINlayer(
    (mlp): ModuleList(
      (0): Linear(in_features=1433, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=1024, out_features=7, bias=True)
    )
  )
  (bn): BatchNorm1d(1433, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [7]:
criterion = nn.CrossEntropyLoss()
lr=0.01
optimizer = optim.SGD(net.parameters(), lr=lr)

%time train(100, data=data, optimizer=optimizer, criterion =criterion, net=net, device=device, val=True)

Epoch: 1 
 Train loss: 1.98614, Train acc:0.12857, Val loss: 1.92502, Val acc:0.28600
Epoch: 2 
 Train loss: 1.58627, Train acc:0.37143, Val loss: 1.66839, Val acc:0.44200
Epoch: 3 
 Train loss: 1.46488, Train acc:0.55714, Val loss: 1.61244, Val acc:0.48000
Epoch: 4 
 Train loss: 1.37898, Train acc:0.68571, Val loss: 1.57231, Val acc:0.49800
Epoch: 5 
 Train loss: 1.30807, Train acc:0.69286, Val loss: 1.53852, Val acc:0.51400
Epoch: 6 
 Train loss: 1.24698, Train acc:0.72143, Val loss: 1.50886, Val acc:0.53200
Epoch: 7 
 Train loss: 1.19317, Train acc:0.74286, Val loss: 1.48202, Val acc:0.53800
Epoch: 8 
 Train loss: 1.14506, Train acc:0.75714, Val loss: 1.45801, Val acc:0.55400
Epoch: 9 
 Train loss: 1.10154, Train acc:0.79286, Val loss: 1.43589, Val acc:0.56400
Epoch: 10 
 Train loss: 1.06193, Train acc:0.80714, Val loss: 1.41520, Val acc:0.57800
Epoch: 11 
 Train loss: 1.02558, Train acc:0.80714, Val loss: 1.39625, Val acc:0.58600
Epoch: 12 
 Train loss: 0.99197, Train acc:0.83571, 